In [2]:
%load_ext autoreload
%autoreload 2

from cnn import CNN
from dyn_cnn import DynCNN
from util import Util
from patch import Patch
from fixer import Fixer
from uglify import Uglify

import cPickle as pickle
import os
import mahotas as mh
import numpy as np
import time

import mlproof as mlp


from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
DOJO_USER_DATA_FOLDER = '/home/d/Dropbox/DOJOSTUDY/'
DOJO_USERS_IDS = [11,13,18,37,40,43,5,50,53,62]
DOJO_USERS = []
for d in DOJO_USERS_IDS:
    DOJO_USERS.append('S'+str(d)+'tif')
    
def read_user(user, slice):
    img = mh.imread(DOJO_USER_DATA_FOLDER+user+os.sep+'z='+str(slice).zfill(8)+'.tif')
    img = Util.crop_by_bbox(img, dojo_bbox)
    return Util.frame_image(img)

def dojo_user_vi(user, slice, gt):
    img = read_user(user, slice)
    return Util.vi(img, gt)

def fix_dojo_slice(cnn, slice, input_image, input_prob, input_rhoana, input_gold,t=0.):
    DOJO_SLICE=slice
    vi_s, merge_pairs, surenesses = Fixer.splits(cnn, 
                                                 input_image[DOJO_SLICE], 
                                                 input_prob[DOJO_SLICE], 
                                                 input_rhoana[DOJO_SLICE], 
                                                 input_gold[DOJO_SLICE],
                                                 sureness_threshold=t,
                                                 oversampling=False)

    best_index = vi_s.index(np.min(vi_s))
    best_vi = vi_s[best_index]
    best_sureness = surenesses[best_index]
    
    return best_vi, best_sureness, vi_s, surenesses

def fix_dojo_slice_user_simulated(cnn, slice, input_image, input_prob, input_rhoana, input_gold, error_rate=0,t=0):
    DOJO_SLICE=slice
    vi_s, merge_pairs, surenesses, good_fixes, bad_fixes = Fixer.splits_user_simulated(cnn, 
                                                 input_image[DOJO_SLICE], 
                                                 input_prob[DOJO_SLICE], 
                                                 input_rhoana[DOJO_SLICE], 
                                                 input_gold[DOJO_SLICE],
                                                 error_rate=error_rate,
                                                 sureness_threshold=t,                                                 
                                                 oversampling=False)
    
    best_index = vi_s.index(np.min(vi_s))
    best_vi = vi_s[best_index]
    best_sureness = surenesses[best_index]
    
    return best_vi, good_fixes, bad_fixes, merge_pairs, vi_s

def threed_vi(gt, seg):
    total_vi = 0
    slice_vi = []    
    for i in range(10):
        current_vi = Util.vi(gt[i].astype(np.int64), seg[i].astype(np.int64))
        total_vi += current_vi
        slice_vi.append(current_vi)
    total_vi /= 10
    return total_vi, slice_vi

def user_threed_vi(gt, user):
    total_vi = 0
    slice_vi = []
    for i in range(10):
        current_vi = dojo_user_vi(user, i, gt[i])
        total_vi += current_vi
        slice_vi.append(current_vi)
    total_vi /= 10
    return total_vi, slice_vi
    
def cnn_threed_vi(cnn, input_image, input_prob, input_rhoana, input_gold):
    total_vi = 0
    slice_vi = []
    slice_sureness = []
    for i in range(10):
        current_vi, sureness = fix_dojo_slice(cnn, i, input_image, input_prob, input_rhoana, input_gold)
        total_vi += current_vi
        slice_vi.append(current_vi)
        slice_sureness.append(sureness)
    total_vi /= 10
    return total_vi, slice_vi, slice_sureness
        

In [3]:
input_image, input_prob, input_gold, input_rhoana, dojo_bbox = Util.read_dojo_data() 

In [4]:
with open('/home/d/nets/MergeNet_larger_border_overlap_cylinder1_dataaugmentation/net.p', 'rb') as f:
    cnn = pickle.load(f)

In [5]:
with open('/home/d/dojo_xp/data/dojo_data_vis2014_patches_predicted.p', 'rb') as f:
    patches = pickle.load(f)
new_patches = []
for p in patches:
    p_new = {}
    p_new['image'] = p['image'] / 255.
    p_new['prob'] = p['prob'] / 255.
    p_new['merged_array'] = p['merged_array'] 
    p_new['border_overlap'] = p['border_overlap'] 
    new_patches.append(p_new)

In [28]:
# fix one slice a time with sureness threshold .95
vol_vi_s = []
for z in range(10):
    vi_s, merge_pairs, surenesses = Fixer.splits(cnn, input_image[z], input_prob[z], input_rhoana[z], input_gold[z], 
                                                 sureness_threshold=.95,
                                                 verbose=False)
    vol_vi_s.append(vi_s[-1])
    

In [29]:
np.median(vol_vi_s)

0.51614654679457939

In [15]:
# simulate user with error rate 0 with limit of N corrections
bigM, out_vol, global_volume = Fixer.splits_global(cnn, input_image, input_prob, input_rhoana, input_gold)

464 generated in 7.46155190468 seconds.
Grouped into 116 patches in 0.00204300880432 seconds.
448 generated in 9.67917203903 seconds.
Grouped into 112 patches in 0.00206303596497 seconds.
456 generated in 9.83008408546 seconds.
Grouped into 113 patches in 0.00214004516602 seconds.
484 generated in 10.1369900703 seconds.
Grouped into 121 patches in 0.0022189617157 seconds.
508 generated in 10.6413669586 seconds.
Grouped into 127 patches in 0.00235080718994 seconds.
488 generated in 10.3497629166 seconds.
Grouped into 122 patches in 0.00232195854187 seconds.
480 generated in 10.1295230389 seconds.
Grouped into 118 patches in 0.00224614143372 seconds.
460 generated in 9.77528500557 seconds.
Grouped into 113 patches in 0.00211215019226 seconds.
488 generated in 10.1580781937 seconds.
Grouped into 120 patches in 0.00228404998779 seconds.
540 generated in 11.1410880089 seconds.
Grouped into 135 patches in 0.00248503684998 seconds.


In [17]:
user_simulated_vi = threed_vi(input_gold, out_vol)
np.median(user_simulated_vi[1])

0.4144289172505391

In [24]:
# random recommendations
bigM, out_vol, global_volume = Fixer.splits_global(cnn, input_image, input_prob, input_rhoana, input_gold, randomize=True)

464 generated in 7.42999410629 seconds.
Grouped into 116 patches in 0.00201487541199 seconds.
448 generated in 9.59283494949 seconds.
Grouped into 112 patches in 0.00198411941528 seconds.
456 generated in 9.68686699867 seconds.
Grouped into 113 patches in 0.0021059513092 seconds.
484 generated in 10.0097439289 seconds.
Grouped into 121 patches in 0.00214195251465 seconds.
508 generated in 10.4890110493 seconds.
Grouped into 127 patches in 0.0022599697113 seconds.
488 generated in 10.2200450897 seconds.
Grouped into 122 patches in 0.00219798088074 seconds.
480 generated in 10.0091488361 seconds.
Grouped into 118 patches in 0.00214195251465 seconds.
460 generated in 9.6586368084 seconds.
Grouped into 113 patches in 0.00209379196167 seconds.
488 generated in 10.0432958603 seconds.
Grouped into 120 patches in 0.00227403640747 seconds.
540 generated in 11.0135400295 seconds.
Grouped into 135 patches in 0.00242280960083 seconds.


In [25]:
random_vi = threed_vi(input_gold, out_vol)
np.median(random_vi[1]) 

0.47030797909077693

In [30]:
# create bigM for the UI tool
bigM, a, global_volume = Fixer.splits_global(cnn, input_image, input_prob, input_rhoana, input_gold)

464 generated in 7.44634389877 seconds.
Grouped into 116 patches in 0.00208497047424 seconds.
448 generated in 9.60027599335 seconds.
Grouped into 112 patches in 0.00206613540649 seconds.
456 generated in 9.67844295502 seconds.
Grouped into 113 patches in 0.00205779075623 seconds.
484 generated in 10.0104978085 seconds.
Grouped into 121 patches in 0.00217604637146 seconds.
508 generated in 10.5294861794 seconds.
Grouped into 127 patches in 0.0022828578949 seconds.
488 generated in 10.2322368622 seconds.
Grouped into 122 patches in 0.0022189617157 seconds.
480 generated in 10.026018858 seconds.
Grouped into 118 patches in 0.00217914581299 seconds.
460 generated in 9.6668651104 seconds.
Grouped into 113 patches in 0.00210499763489 seconds.
488 generated in 10.0534870625 seconds.
Grouped into 120 patches in 0.00218987464905 seconds.
540 generated in 11.0391199589 seconds.
Grouped into 135 patches in 0.00241494178772 seconds.


In [31]:
with open('/home/d/dojo_xp/data/bigM_new_cnn.p', 'wb') as f:
    pickle.dump(bigM, f)

In [32]:
from uitools import UITools

In [33]:
merge_errors = UITools.get_top5_merge_errors(cnn, input_image, input_prob, input_rhoana)

working on slice 0
working on slice 1
working on slice 2
working on slice 3
working on slice 4
working on slice 5
working on slice 6
working on slice 7
working on slice 8
working on slice 9
merge error correction done after 1108.03680515 seconds


In [34]:
with open('/home/d/dojo_xp/data/merges_new_cnn.p', 'wb') as f:
    pickle.dump(merge_errors, f)

In [40]:
len(merge_errors[4][3])

5

In [47]:
with open(os.path.join('/home/d/dojo_xp/data/ui_out/new_cnn_test_with_saving_times/times.p'), 'rb') as f:
    c_t = pickle.load(f)
    c_t = [int(v) for v in c_t]
np.mean(c_t)

1889.0